<a href="https://colab.research.google.com/github/Ranjana-Kambhammettu/password-generator-and-analyser/blob/main/Password_Generator_and_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collecting Common Passwords and storing them in a csv file

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/")
!ls

MyDrive


In [3]:
!pip install Distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Distance: filename=Distance-0.1.3-py3-none-any.whl size=16275 sha256=65b642eaa12f10ba3a45c011d99ef52db20199e39ecb21dc1e055ea1aeeba4b0
  Stored in directory: /root/.cache/pip/wheels/fb/a8/64/6edcab63ec51512a87cacf9b3563c711ad6b7b05d61b704493
Successfully built Distance


In [4]:
import os
os.chdir("/content/drive/My Drive/Password Generator and Analyzer")
!ls

 common_passwords.csv  'Password Generator and Analyzer.ipynb'
'Leaked Database'


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from nltk.metrics import edit_distance
#import Pycluster as PC
#from pyclustering.cluster.kmedoids import kmedoids

In [9]:
# Scraping Website 1
url1 = "https://en.wikipedia.org/wiki/Wikipedia:10,000_most_common_passwords"
webdata  = requests.get(url1).text
soup = BeautifulSoup(webdata,"html.parser")
cols = soup.find_all('div',class_='div-col')
data = {"Common Passwords":[]}
for col in cols:
    passwds=col.find_all('li')
    for passwd in passwds:
        if passwd.text.strip()!='':
            data["Common Passwords"].append(passwd.text)
#Scraping Website 2
url2 = 'https://github.com/OWASP/passfault/blob/master/wordlists/wordlists/10k-worst-passwords.txt'
webdata  = requests.get(url2).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_="blob-code blob-code-inner js-file-line")
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)
#Scraping Website 3
url3='https://github.com/danielmiessler/SecLists/blob/master/Passwords/Common-Credentials/10-million-password-list-top-100000.txt'
webdata = requests.get(url3).text
soup = BeautifulSoup(webdata,"html.parser")
passwds = soup.find_all('td',class_='blob-code blob-code-inner js-file-line')
for passwd in passwds:
    data["Common Passwords"].append(passwd.text)    

#Writing all unique passwrods into a csv file    
common_password = pd.DataFrame.from_dict(data)
common_password = common_password[~common_password['Common Passwords'].str.isdigit()]
common_password = common_password.drop_duplicates(subset='Common Passwords', keep="first")
common_password = common_password.reset_index(drop=True)
common_password = common_password.sample(frac=1)
common_password.to_csv('common_passwords.csv')
words=common_password['Common Passwords'].tolist()
print(len(words))

62263


In [20]:
#clustering using AffinityPropagation
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
words=common_password['Common Passwords'].tolist()
words=words[:4000]
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
clusters=[]
for cluster_id in np.unique(affprop.labels_):
    clus=[]
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    clus.append(exemplar)
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    for p in cluster:
      if p!=exemplar:
        clus.append(p)
    clusters.append(clus)
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *shadow:* 7hrdnw23, sexdog, sexnow, shadow, shadow14, shadow3, shaqfu, showoff, tashadog
 - *dima1991:* alina1997, dima1990, dima1991, ducati91, gfif1991
 - *punjab:* SuzjV8, buhjvfybz, dunlap, punched, pundai, punjab, punjabi
 - *SERGEY:* BUCKEYES, MARCUS, PEEWEE, SERGEY, SPANKY, STEPHEN, STINGER, STORMY, WEASEL, WESLEY, hpSALGaY
 - *1234abc:* 123456abc, 12345zxcv, 1234abc, 123QWEasd, 123abcd, 19371ayj, 2348TYty
 - *dfnheirf:* dfnheirf, dtcyeirf, gtnheif
 - *dps140786:* dps140786
 - *djhjyf:* 8218yxfz, djdjdj, djhjyf, ePVjb6, kjkbnf, rjnjgtc, tujhjdf, vfhujif, vfvjyn
 - *norway:* g0away, norwalk, norway, noway1, nurlan, parkway, railway, thursday
 - *nEMvXyHeqDd5OQxyXYZI:* nEMvXyHeqDd5OQxyXYZI
 - *josh123:* 123w123, asshole123, gtnhj123, jesse123, jorge123, josh123, joshua2, noah123, pass123, pooh123, sas123, toshiba
 - *amsterda:* amsterda, amstrad, azteca, mierda, yesterda
 - *alabama:* akademia, alabama, alabaste, aldebaran, alfaromeo, bammbamm, hayabusa, hollaback, kalamata, ka

In [22]:
clusters #Nested list in which each element contains cluster of passwords
temp=clusters #creating a copy of clusters to work on

In [26]:
#Finding different patterns present in each cluster and the frequency of these patterns
from difflib import SequenceMatcher 
def longest_Substring(s1,s2): 
     seq_match = SequenceMatcher(None,s1,s2) 
     match = seq_match.find_longest_match(0, len(s1), 0, len(s2)) 
     # return the longest substring 
     if (match.size!=0): 
          return (s1[match.a: match.a + match.size])  
     else: 
          return False
patternInClusters=[]
for i in range(len(temp)):
  patternInClusters.append({})
  exemplar=temp[i][0]
  for j in temp[i][1:]:
    x=longest_Substring(exemplar,j)
    if x!=False and len(x)>1:
      if x in patternInClusters[i]:
        patternInClusters[i][x]+=1
      else:
        patternInClusters[i][x]=1

patternInClusters

[{'do': 1, 'ow': 1, 'shadow': 2, 'sha': 1, 'sh': 1, 'shado': 1},
 {'a199': 1, 'dima199': 1, '91': 1, '1991': 1},
 {'un': 1, 'pun': 2, 'punjab': 1},
 {'EY': 2, 'ER': 1, 'SE': 1},
 {'1234': 2, '123': 2, '234': 1},
 {'eirf': 1, 'nhei': 1},
 {},
 {'dj': 1, 'jhj': 1, 'jy': 1},
 {'way': 4, 'norwa': 1, 'ay': 1},
 {},
 {'123': 7, 'josh': 1, 'h123': 2, 'osh': 1},
 {'amst': 1, 'te': 1, 'erda': 1, 'sterda': 1},
 {'alaba': 1, 'al': 2, 'bam': 1, 'ab': 1, 'laba': 1, 'ala': 2, 'la': 1},
 {'jordan': 2, 'ord': 1, 'n01': 1, 'an01': 1, 'or': 1},
 {'hol': 1, 'ho': 1, 'cho': 2, 'nic': 1, 'ni': 1, 'ol': 2, 'ch': 1},
 {'evild': 1,
  'evi': 1,
  'evil': 1,
  'dead': 1,
  'ad': 1,
  'ea': 1,
  'vil': 1,
  'ld': 1},
 {'000': 2, '0000': 1},
 {'blu': 2, 'blue': 5, 'blue2': 1, '20': 1},
 {'aaaaa': 2, 'aaa': 3, 'a2': 3},
 {'str': 2, 'tr': 1, 'ren': 1, 'ng': 1},
 {'a1': 1,
  'ic': 1,
  'on': 1,
  'ica1': 1,
  'ni': 1,
  'ica': 1,
  'moni': 2,
  'onic': 1},
 {'er': 5, 'der': 2, 'yd': 1, 'ryder': 1},
 {'ojo11': 1, 'mo

In [29]:
#adding the most common patterns in each cluster to a final pattern list
patterns=[]
for sampleDict in patternInClusters:
  if sampleDict=={}:
    continue
  itemMaxValue = max(sampleDict.items(), key=lambda x: x[1])
  #print('Maximum Value in Dictionary : ', itemMaxValue[1])
  listOfKeys = list()
  # Iterate over all the items in dictionary to find keys with max value
  for key, value in sampleDict.items():
    if value == itemMaxValue[1]:
      listOfKeys.append(key)
  for i in listOfKeys:
    if i not in patterns:
      patterns.append(i)
patterns

['shadow',
 'a199',
 'dima199',
 '91',
 '1991',
 'pun',
 'EY',
 '1234',
 '123',
 'eirf',
 'nhei',
 'dj',
 'jhj',
 'jy',
 'way',
 'amst',
 'te',
 'erda',
 'sterda',
 'al',
 'ala',
 'jordan',
 'cho',
 'ol',
 'evild',
 'evi',
 'evil',
 'dead',
 'ad',
 'ea',
 'vil',
 'ld',
 '000',
 'blue',
 'aaa',
 'a2',
 'str',
 'moni',
 'er',
 'ojo11',
 'mo',
 'mojo',
 '11',
 'zxc',
 'zxcv',
 'love',
 'br',
 'bryan',
 'an',
 'azz',
 'ova',
 'lan',
 'rate',
 'espera',
 'esper',
 'ca',
 'on',
 'su',
 'ge',
 'rna',
 'rn',
 'fur',
 'ur',
 'ace',
 'butt',
 'bur',
 'ina',
 'jack',
 'auto',
 'ill',
 'tor',
 'ector',
 'ecto',
 'or',
 'ect',
 've',
 'cto',
 'ctor',
 'and',
 'fre',
 '12',
 'hot',
 'ot',
 'cial',
 'pecial',
 'pe',
 'pi',
 '19',
 'oleg1',
 'shir',
 'sh',
 'man',
 'da',
 'dais',
 'ab',
 'ba',
 'angel',
 'arky',
 'ra',
 'ar',
 'ake',
 'ma',
 '12345678',
 'cooter',
 'co',
 'scooter',
 '6chevy',
 'killer',
 'qwerty',
 'ggy',
 'master',
 'ers',
 'head',
 'david',
 'ing',
 'ga',
 'lon',
 'ck',
 'dick',
 '

In [6]:
#Implementing Algorithm to generate passwords
import random
words = input("Enter five words(give space between each word): ").split()
numbers = input("Enter two numbers(give space between each number): ").split()
num_passwords = int(input("Enter number of passwords required:"))
special_chars = ['@','$','!','#','%','&','(',')','0','3','8','<','|']
punc = ['@','*','+','-',':','”','/','\\','~','?','[',']','{','}','$','!','#','%','&','(',')','_' ,'<','|']
alphabets = ['a','s','i','r','x','q','c','j','o','e','b','k','l']
c2s = {}
for i in range(13):
  c2s[alphabets[i]]=special_chars[i]
def randomly_capitalize(s):
  result = ''
  for char in s:
      flag = random.randint(0, 1)
      if flag == 0:
          result += char.upper()
      else:
          result += char.lower()
  return result
def merge_randomly(str1,str2):
  result = ''
  flag = random.randint(0, 1)
  if flag == 0:
      result = str1 + str2
  else:
      result = str2 + str1
  return result,flag
def charToSpecial(s):
  for char in s:
    if char in c2s:
      s = s.replace(char,c2s[char])
  return s
def insert_number(s,n,f,str1,str2):
  flag = random.randint(0, 2)
  if flag == 0:
    return n+s
  elif flag == 2:
    return s+n
  else:
    if f==0:
      return s[:len(str1)]+n+s[len(str1):]
    else:
      return s[:len(str2)]+n+s[len(str2):]
def add_punctuation(s):
  return s+random.choice(punc)
passwords_generated=[]
while num_passwords>0:
  temp = list(words)
  string1 = random.choice(temp)
  temp.remove(string1)
  string2 = random.choice(temp)
  number = random.choice(numbers)
  string1,string2 = randomly_capitalize(string1),randomly_capitalize(string2)
  final_string,f = merge_randomly(string1,string2)
  final_string = charToSpecial(final_string)
  final_string = insert_number(final_string,number,f,string1,string2)
  final_string = add_punctuation(final_string)
  C = len(final_string)
  if C < 8:
    for i in range(8 - C):
      final_string += random.choice(special_chars)
  passwords_generated.append(final_string)
  print("Final Passowrd:",final_string)
  num_passwords -= 1
  

Enter five words(give space between each word): dog cat fish horse bird
Enter two numbers(give space between each number): 45 67
Enter number of passwords required:4
Final Passowrd: f!$hD0g67_
Final Passowrd: 67dOgF!SH*
Final Passowrd: (At678!RD”
Final Passowrd: B!Rd67h0R$E”


In [9]:
#To analyze given/generated password
import re
patterns=['shadow', 'a199', 'dima199', '91', '1991', 'pun', 'EY', '1234', '123', 'eirf', 'nhei', 'dj', 'jhj', 'jy', 'way', 'amst', 'te', 'erda', 'sterda', 'al', 'ala', 'jordan','cho',
 'ol', 'evild', 'evi','evil','dead','ad','ea','vil','ld','000','blue','aaa','a2','str','moni','er','ojo11','mo','mojo','11','zxc','zxcv','love','br','bryan',
 'an','azz','ova','lan','rate','espera','esper','ca','on','su','ge','rna','rn','fur','ur','ace','butt','bur','ina','jack','auto','ill','tor','ector',
 'ecto','or','ect','ve','cto','ctor','and','fre','12','hot','ot','cial','pecial','pe','pi','19','oleg1','shir','sh','man','da','dais','ab','ba','angel',
 'arky','ra','ar','ake','ma','12345678','cooter','co','scooter','6chevy','killer','qwerty','ggy','master','ers','head','david','ing','ga','lon','ck',
 'dick','long','ng','count','ou','oun','ted','ire','oo','fire','artin','martin','in1','ker1','acker','23','poppy','pp','ppy123','be','zer','ee','thunder','dan',
 'ana','ho','ota','tac','vol','el','elvis','s99','elvi','ri','alex','se','hi','pimp','ER','lo','sasha1','ch','rest','john','sta','out','st','ne','elcome','we',
 'welcome','wel','nat','mar','ball','tymrf','ger','ki','king','es','ro','happy','ell','flash','qwe','all','12345','th','sa','daniel','ani','shar','jerr',
 'ken','one','fi','fir','firstone','624','ac','mic','art','sha','superm','dd','ista','rist','2000','at','scott','in','indi','bz','hfc','987654321','54321',
 'ton','fgf','gf','fg','kocha','oc','land','home','s1','good','iper','pers','ipers','per','vi','viper','ett','et','ob','bb','obby','y1','bby1','rob',
 'robb','robby','nokia','batman','aezakmi','a123','izard','fuck','min','esca','get','gettysbu','nder','ion','mi','ud','rude','rud','ru','boy','mari','emon',
 'big','rf','ens','green','gre','ford','ord','kam','bla','blast','bl','ster','laster','las','111','ham','ture','lar','ola','solar','sol','so','tn','ghbdtn',
 'ghb','bd','dtn','nbr','nb','ey','rand','mme','mmer','tim','ti','im','mer','immer','sun','llin','ossi','lu','zxcvb','jxrf','ama','mada','ada','sex','ver',
 'ter','hel','dragon','a19','95','199','1995','vova199', 'vova', 'purple', 'cat', 'fucks', 'tan', 'black', 'RE', 'che', 'access', 'dolphin', 'hin',
 'ret', 'barr', 'je', 'mou', 'oune', 'll', 're', 'ant', 'bu', 'bal', 'ian', 'anthon', 'on1', 'nt', 'nth', 'anth', '1Cccccc', 'Ccccc', 'atthew', 
 'matt', 'ja', 'am','sam', 'ock', 'sea', 'oliver', 'oliv', 'er2', 'itsu', 'piderman', 'ide', 'dee', 'smith', 'nch', 'no00', 'benno', 'cks', 'ero', 'us', 
 'ven', 'a2010', '55555', 'secret', 'swe', 'td', 'eanin', 'soccer', 'un', 'junior', 'age', 'verag', 'tig', 'mother', 'smo', 'the', 'her', 'moth', 
 'fgh', 'nin', 'harley', 'ent', 'red', 'ward', 'er1', 'sana', 'os', 'hea', 'he', 'ik', 'kik', 'kok', 'con', 'puss', 'rocket', 'rock', 'cola', 'col', 
 '00', 'dle', 'mod', 'ov', 'casa', 'asanov', 'anov', 'anova', 'eni', 'is', 'is1', 'ight', 'averic', 'eric', 'ric', 'maveric', 'CO', 'OC', 'win', 
 'ed', 'gan', 'allard', 'eg', 'ta', 'ine', 'bel', 'lous', 'horny', 'hor', 'yme', 'me', 'nascar', 'andalf', 'gandalf', 'dal', 'li', 'april2','147', 
 'ile', 'le', 'fu', 'ku', 'ya', 'yak', 'ip', 'ul', 'andy', 'ena', 'girl', 'lexandr', 'music', 'pas', 'bob', 'lovejo', 'mike', 'password', 'passw', 
 'ey2010', 'ashi', 'itch', 'my', 'schnu', 'nuff', 'la', 'thug', 'four', 'ble', 'ghjg', 'ghjghj', 'ghj', 'stuff', 'ove', 'qaz', 'mmy', 'birthday',
  'pol', 'ook', 'eagle', '2wsx', 'par', 'wayne', 'apple', 'DICK', 'IC', 'hris', 'anger', 'tcast', 'tca', 'tcas', 'ban', 'ustin', 'nchester',
   'field', 'olf','silver', 'xxx', 'dra', 'rat', 'bull', 'amir', 'mon', 'rus', 'ito', 'gri', 'negri','negrit','negr','hilli','chill','chillin',
   'hello','night','ic']
# Function to categorize password
def password(v):
    # the password should not be a
    # newline or space
    if v == "\n" or v == " ":
        return "Password cannot be a newline or space!"
    # the password length should be in
    # between 9 and 20
    if 9 <= len(v) <= 20:
        #checks for patterns in password
        for pattern in patterns:
          if pattern in v:
            return "Weak Password: Weak patterns have been recognized in the password"
        # checks for occurrence of a character 
        # three or more times in a row
        if re.search(r'(.)\1\1', v):
            return "Weak Password: Same character repeats three or more times in a row"
        # checks for occurrence of same string 
        # pattern( minimum of two character length)
        # repeating
        if re.search(r'(..)(.*?)\1', v):
            return "Weak password: Same string pattern repetition"
        else:
            return "Strong Password!"
    else:
        return "Password length must be 9-20 characters!"
#Analyzing strength of generated passwords: 
for p in passwords_generated:
 print(password(p))
#Analyzing strength of password given by user: 
print(password('blackPanther45'))


Strong Password!
Strong Password!
Strong Password!
Strong Password!
Weak Password: Weak patterns have been recognized in the password


In [ ]:
#Storing Breached passwords from local file into a MySql Database
import pandas as pd
import mysql.connector
# Connect to the MySQL database
#Creating a databse.Note: The same code wont run the second time onwards as the database would have been already created in the first run
#connect=mysql.connector.connect(user='root', password='password',
#                              host='localhost')
#cursor=connect.cursor()
#cursor.execute("CREATE DATABASE breachedpasswords")
#Creatimg a table.Note: Similar to creating a database, creating a table can only be done once.Afterwards, the same code wont run as the table would have been already created
#connect=mysql.connector.connect(user='root', password='password',
#                              host='localhost',database="breachedpasswords")
#cursor=connect.cursor()
#cursor.execute("CREATE TABLE passwords(Password longtext)")
connect=mysql.connector.connect(user='root', password='password',
                              host='localhost',database="breachedpasswords")
cursor=connect.cursor()                              
df = pd.read_csv("file path to the csv file",encoding='latin1', header=None, names=['Password'])
df = df.fillna('')
for row in df.itertuples():
    cursor.execute("""
        INSERT INTO passwords(Password)
        VALUES (%s)
    """, (row.Password,))
connect.commit()
connect.close()